# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

In [2]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [9]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: AI-powered tools can generate high-quality content such as product descriptions, social media posts, and blog articles, reducing the need for human writers and editors.
2. **Product Design**: Generative AI can create 3D models, prototypes, and even entire product lines, streamlining the design process and reducing costs.
3. **Marketing Automation**: AI-powered chatbots and virtual assistants can automate customer support, lead generation, and sales outreach, improving customer experience and efficiency.
4. **Data Analysis**: Generative AI algorithms can analyze large datasets, identify patterns, and generate insights, enabling data-driven decision-making.
5. **Predictive Maintenance**: AI-powered predictive maintenance tools can analyze equipment performance data to predict potential failures, reducing downtime and increasing overall efficiency.
6. **Financial Modeling**: Ge

## Alternative approach - using OpenAI python library to connect to Ollama

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [27]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")


Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [13]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

In [16]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [18]:
summarize("https://edwarddonner.com")

"**Summary of Edward Donner's Website**\n=====================================\n\n### Overview\n\nThis website is a personal blog written by Ed, the co-founder and CTO of Nebula.io. The site showcases his passion for writing code, experimenting with Large Language Models (LLMs), and AI.\n\n### Recent News and Announcements\n---------------------------\n\n*   **January 23, 2025:** LLM Workshop – Hands-on with Agents – resources\n*   **December 21, 2024:** Welcome, SuperDataScientists!\n*   **November 13, 2024:** Mastering AI and LLM Engineering – Resources\n*   **October 16, 2024:** From Software Engineer to AI Data Scientist – resources\n\n### Personal Information\n\nEd is the co-founder and CTO of Nebula.io, a company applying AI to help people discover their potential. He has also founded and CEO'd an AI startup that was acquired in 2021.\n\n### Contact Information\n\n*   Email: [ed at] edwarddonner [dot] com\n*   Website: www.edwarddonner.com\n*   Social Media:\n    *   LinkedIn\n  

In [19]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [20]:
display_summary("https://edwarddonner.com")

# Website Summary
### **Home**
The website is a personal blog of Edward Donner, the co-founder and CTO of Nebula.io. The content discusses his interests in writing code, LLMs, DJing, and amateur electronic music production.

### **News/Announcements**

* **LLM Workshop – Hands-on with Agents**: A workshop scheduled for January 23, 2025.
* **Welcome, SuperDataScientists!**: An announcement posted on November 13, 2024.
* **Mastering AI and LLM Engineering – Resources**: Resources available for mastering AI and LLM engineering.
* **From Software Engineer to AI Data Scientist – resources**: Additional resources for transitioning from software engineer to AI data scientist.

### **Other**

The website is a personal blog with a mix of personal thoughts, professional updates, and industry-related news. It seems to be a platform for Edward Donner to share his experiences and insights in the field of LLMs and AI.

In [21]:
display_summary("https://www.theregister.com/2025/03/13/openai_data_copyright/")

OpenAI, the company behind the popular chatbot ChatGPT, has submitted a response to the White House's call for public input on its proposed action plan for artificial intelligence (AI) development and deployment.

The plan aims to promote the responsible use of AI, ensure that it benefits society as a whole, and prevent any potential negative consequences. OpenAI is proposing several key measures to achieve these goals, including:

1. **Fair use defenses**: Allowing developers to use large language models like ChatGPT for research, education, and other purposes without infringing on copyright or violating data-mining exceptions.
2. **Data-mining exceptions**: Permitting the use of AI training data in certain circumstances, such as when it is necessary for research or development.
3. **AI Economic Zones**: Creating designated areas for the development and deployment of AI infrastructure, including solar arrays, wind farms, and nuclear reactors.
4. **Streamlined permitting processes**: Reducing bureaucratic hurdles for developers seeking to build AI-powered projects, including easing accreditation processes and shortening testing authorities.
5. **Federal agency adoption**: Encouraging federal agencies to adopt AI tools and technologies, and removing barriers to their use.

OpenAI is also proposing the creation of "AI Growth Zones" - designated areas where companies can innovate and develop new AI-powered products and services without worrying about regulatory hurdles.

Google has also responded to the White House's call for public input, arguing that fair use defenses and data-mining exceptions are necessary to allow developers to work with large language models like ChatGPT.

The move is seen as an effort by OpenAI to shape the future of AI development and deployment in the United States, and to ensure that it benefits society as a whole.